In [3]:
from src.Dataset import getDataLoader
import torch

In [4]:
classes = ["4ask","8pam","16psk","32qam_cross","2fsk","ofdm-256"]
iq_samples = 1024
samples_per_class= 1000
batch_size = 32
epochs = 10
eff_net_PATH = f'src/SavedModels/efficientNet_epochs_{epochs}.pt'
in_channels = 2

dl_train, ds_train, dl_test, ds_test, dl_val, ds_val = getDataLoader(
    classes = classes,
    iq_samples = iq_samples,
    samples_per_class= samples_per_class,
    batch_size=batch_size
    )

In [5]:
ds_train[0][0].shape

(2, 1024)

In [6]:
next(iter(dl_train))[0].shape

torch.Size([32, 2, 1024])

In [25]:
model = torch.load('src/SavedModels/VQVAE_epochs_10.pt').to('cuda')

/tmp/ipykernel_783178/920308708.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('src/SavedModels/VQVAE_epochs_10.pt').to('cuda')


In [32]:
data = torch.cat([ model.codebook.quantize_indices(model.encode(x.to('cuda'))) for x,label in  dl_train ])

In [36]:
data_label = []

for x, label in dl_train:
    # Quantize indices
    quantized_indices = model.codebook.quantize_indices(model.encode(x.to('cuda')))  # Shape: (32, 512)

    # Ensure label is of shape (32, 1)
    label = label.unsqueeze(1)  # Shape: (32, 1)

    # Concatenate label with quantized indices
    combined_tensor = torch.cat((label.to('cuda'), quantized_indices), dim=1)  # Shape: (32, 513)

    # Append the combined tensor to data_label list
    data_label.append(combined_tensor)

# After the loop, convert data_label list to a single tensor if needed
data_label_tensor = torch.cat(data_label, dim=0)  # Shape: (batch_size * num_batches, 513)

# Print shape to verify
print(data_label_tensor.shape)


torch.Size([6000, 513])


In [44]:
data_label = torch.cat([torch.cat((label.unsqueeze(1).to('cuda'), model.codebook.quantize_indices(model.encode(x.to('cuda')))), dim=1) for x, label in dl_train],dim=0)


In [45]:
data_label.shape

torch.Size([6000, 513])